In [70]:
# Load libraries
import numpy as np
import pandas as pd
import io

#  Mock Midterm Exercise

In this exercise you will have to implement code in the sections inside *Fill with Your Code* (*Load Data*, *Data Preprocessing* and *Create Model*) create a model to predict the column *exitus* in the dataset given. The dataset is already split into train, validation, and test subsets. To see to which subset belong each observation, you need to check the *dataset* column.

The code that is already written in this notebook **CANNOT BE CHANGED**. You can only add code in the *Fill with Your Code* section.

You must achieve in the last cell of this notebook an **AUC over test of at least 0.93**.

# Fill With Your Code

### [1] Load Data

In [71]:
dat = pd.read_csv('../data/dat.csv', sep = ",")

In [72]:
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus,dataset
0,2016-01,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,0,train
1,2016-01,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,0,train
2,2016-01,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,0,train
3,2016-01,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,0,train
4,2016-01,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...
32701,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0,test
32702,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0,test
32703,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0,test
32704,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0,test


### [2]  Data Preprocessing

In [73]:
dat.drop('date', axis = 1, inplace = True)

In [74]:
cat_var = ['severity', 'ambulatory', 'origin', 'tip_grd', 'tip_adm']
non_cat_var = list(set(dat.columns) - set(cat_var))
num_var = list(set(dat.columns) - set(cat_var) - {'dataset', 'exitus'})

In [75]:
dat.isna().any()

severity            True
mortality_ratio     True
age                 True
num_proc            True
ambulatory          True
origin              True
expected_length     True
tip_grd             True
tip_adm             True
exitus             False
dataset            False
dtype: bool

In [76]:
from fancyimpute import IterativeImputer as MICE # pip install fancyimpute
# 3) Define "model"
model = MICE()

# 4) Train "model"
model.fit(dat[num_var][dat['dataset'] == 'train'])

# 5) "Predict"
dat[num_var] = model.transform(dat[num_var])
dat.isna().any()

severity            True
mortality_ratio    False
age                False
num_proc           False
ambulatory          True
origin              True
expected_length    False
tip_grd             True
tip_adm             True
exitus             False
dataset            False
dtype: bool

In [77]:
dat[cat_var][dat['dataset'] == 'train'].isna().sum()

severity      14491
ambulatory    21528
origin        14359
tip_grd        9492
tip_adm        4255
dtype: int64

In [78]:
dat.loc[dat['dataset'] == 'train', cat_var] = dat.loc[dat['dataset'] == 'train', cat_var].fillna('UNKNOWN')
dat[cat_var][dat['dataset'] == 'train'].isna().sum()

severity      0
ambulatory    0
origin        0
tip_grd       0
tip_adm       0
dtype: int64

In [79]:
dat[cat_var] = dat[cat_var].astype('str')

In [80]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output = False, drop='first')

# 4) Training model
ohe.fit(dat[cat_var][dat['dataset'] == 'train'])

# 5) Predicting
dat_ohe = pd.DataFrame(ohe.fit_transform(dat[cat_var]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()
dat = pd.concat((dat[non_cat_var], dat_ohe), axis=1)

In [81]:
dat

,exitus,num_proc,mortality_ratio,age,expected_length,dataset,severity_2.0,severity_3.0,severity_4.0,severity_UNKNOWN,...,origin_9.0,origin_UNKNOWN,origin_nan,tip_grd_Q,tip_grd_UNKNOWN,tip_grd_nan,tip_adm_2.0,tip_adm_3.0,tip_adm_UNKNOWN,tip_adm_nan
0,0,21.0,0.408730,12596.0,151.0,train,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,22.0,0.306931,20973.0,99.0,train,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,19.0,0.278481,19611.0,87.0,train,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,22.0,0.150289,13583.0,100.0,train,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,2.0,0.016573,18042.0,44.0,train,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32701,0,2.0,0.028365,23619.0,2.0,test,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32702,0,1.0,0.000606,3935.0,2.0,test,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32703,0,4.0,0.040452,30163.0,2.0,test,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32704,0,4.0,0.000000,29012.0,0.0,test,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### [3] Create Model

In [82]:
from sklearn.ensemble import RandomForestClassifier as model_constructor
from sklearn.metrics import roc_auc_score as metric

In [83]:
# Random Forest
n_estimators_values = [10, 100, 1000]
max_features_values = [2, 5, 10]
max_samples_values = [10, 100, 1000, dat[dat['dataset'] == 'train'].shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}

In [84]:
num_iter = 1
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteration = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].exitus.values)


                        # [5] Predict
                        pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
                        pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!

                        # [6] Compute metric
                        metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
                        metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1



Iteration = 1
Metric train = 0.49 - Metric validation = 0.50.
Iteration = 2
Metric train = 0.86 - Metric validation = 0.73.
Iteration = 3
Metric train = 0.89 - Metric validation = 0.83.
Iteration = 4
Metric train = 1.00 - Metric validation = 0.81.
Iteration = 5
Metric train = 0.88 - Metric validation = 0.86.
Iteration = 6
Metric train = 0.93 - Metric validation = 0.92.
Iteration = 7
Metric train = 0.96 - Metric validation = 0.93.
Iteration = 8
Metric train = 1.00 - Metric validation = 0.92.
Iteration = 9
Metric train = 0.93 - Metric validation = 0.92.
Iteration = 10
Metric train = 0.94 - Metric validation = 0.93.
Iteration = 11
Metric train = 0.97 - Metric validation = 0.93.
Iteration = 12
Metric train = 1.00 - Metric validation = 0.93.
Iteration = 13
Metric train = 0.56 - Metric validation = 0.55.
Iteration = 14
Metric train = 0.88 - Metric validation = 0.88.
Iteration = 15
Metric train = 0.90 - Metric validation = 0.85.
Iteration = 16
Metric train = 1.00 - Metric validation = 0.84.
I

In [85]:
grid_results = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model = grid_results.iloc[0]
best_model

max_features      10.000000
n_estimators    1000.000000
max_samples     1000.000000
metric_train       0.962544
metric_val         0.937265
Name: 35, dtype: float64

In [86]:
model = model_constructor(max_features=int(best_model['max_features']),
                          n_estimators=int(best_model['n_estimators']),
                          max_samples=int(best_model['max_samples']),
                          random_state=0)

In [87]:
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)
var_imp = pd.DataFrame({'var': dat.drop(['exitus', 'dataset'], axis = 1).columns, 'imp': model.feature_importances_})
var_imp.sort_values(['imp'], ascending = False, inplace = True)
var_imp

,var,imp
1,mortality_ratio,0.329915
2,age,0.233788
3,expected_length,0.148166
0,num_proc,0.121442
21,tip_grd_UNKNOWN,0.027640
18,origin_UNKNOWN,0.027140
6,severity_4.0,0.023575
25,tip_adm_UNKNOWN,0.019058
7,severity_UNKNOWN,0.017575
5,severity_3.0,0.016779


In [88]:
# [4] Train model
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)

RandomForestClassifier(max_features=10, max_samples=1000, n_estimators=1000,
                       random_state=0)

In [89]:
print(var_imp.nlargest(10, 'imp')['var'].tolist())

['mortality_ratio', 'age', 'expected_length', 'num_proc', 'tip_grd_UNKNOWN', 'origin_UNKNOWN', 'severity_4.0', 'tip_adm_UNKNOWN', 'severity_UNKNOWN', 'severity_3.0']


In [90]:
dat = dat[['mortality_ratio', 'age', 'expected_length', 'num_proc', 'tip_grd_UNKNOWN', 'origin_UNKNOWN', 'severity_4.0', 'tip_adm_UNKNOWN', 'severity_UNKNOWN', 'severity_3.0', 'exitus', 'dataset']]

In [91]:
model.fit(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].exitus.values)
# [5] Predict
pred_train = model.predict_proba(dat[dat['dataset'] == 'train'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_val = model.predict_proba(dat[dat['dataset'] == 'val'].drop(['exitus', 'dataset'], axis = 1)) # predict!
pred_test = model.predict_proba(dat[dat['dataset'] == 'test'].drop(['exitus', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train[:,1])
metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val[:,1])
metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test[:,1])

# Evaluate Model

In [92]:
# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.96 - Metric val = 0.94 - Metric test = 0.94
